In [1]:
import torch
import pandas as pd
import numpy as np
import pickle
import HMM_Likelyhood
import plotly.express as px

with open('DataPhase.pickle','rb') as f:
    X = pickle.load(f) 
X_tensor = X_tensor = torch.from_numpy(X).to(torch.float64)
#X_tensor = torch.reshape(X_tensor.T,(-1,90,330))
torch.set_default_dtype(torch.float64)

def Accumulated_HHM_LL(X: Tensor,
    Pinit: Tensor,
    kappa: Tensor,
    mu: Tensor,
    Tk: Tensor,
    n: int,
    p: int,
    K: int) -> Tensor:
  _0 = __torch__.HMM_Likelyhood.HMM_log_likelihood
  kappa_con = torch.minimum(^<python_value>()(kappa), torch.tensor([800]))
  _1 = torch.sqrt(torch.sum(torch.pow(mu, 2), [0]))
  mu_con = torch.div(mu, _1)
  Subjectlog_Likelihood = torch.zeros([n])
  for subject in range(n):
    _2 = torch.slice(X)
    _3 = torch.mul(330, subject)
    _4 = torch.mul(330, torch.add(subject, 1))
    _5 = torch.slice(_2, 1, _3, _4)
    _6 = torch.select(torch.slice(Pinit), 1, subject)
    _7 = ^<python_value>()(_6)
    _8 = ^<python_value>()(torch.select(Tk, 0, subject))
    _9 = _0(_5, _7, kappa_con, mu_con, _8, p, K, )
    _10 = torch.select(Subjectlog_Likelihood, 0, subject)
    _11 = torch.copy_(_10, _9)
  return torch.sum(Subjectlog_Likelihood)



In [2]:
n_iters = 1000
K=7
p=90
Subjects = 10
LearningRates = [0.1,0.5,0.8,1,1.2,1.5]
Likelihoods = np.zeros((len(LearningRates),n_iters))

for idx,lr in enumerate(LearningRates):

    kappa,mu,Tk,Pinit = HMM_Likelyhood.InitializeParameters(Subjects,p,K)
    print(lr)
    Parameters = [
        {'params':Pinit},
        {'params':kappa},
        {'params':mu},
        {'params':Tk},
    ]

    Adam = torch.optim.Adam(Parameters,lr=lr)
    Likelihoods[idx,:] = HMM_Likelyhood.OptimizationTraj(X_tensor[:,0:330*Subjects],[Pinit,kappa,mu,Tk],lose=HMM_Likelyhood.Accumulated_HHM_LL,Optimizer=Adam,n=Subjects,n_iters=n_iters,K=K)

0.1
epoch 1; Log-Likelihood = -241267.75467613363
epoch 101; Log-Likelihood = -252274.22439161033
epoch 201; Log-Likelihood = -263583.49965616694
epoch 301; Log-Likelihood = -274323.1023210049
epoch 401; Log-Likelihood = -283339.91654554894
epoch 501; Log-Likelihood = -289772.1189656607
epoch 601; Log-Likelihood = -293427.64407337684
epoch 701; Log-Likelihood = -295356.7012750602
epoch 801; Log-Likelihood = -296377.20911690395
epoch 901; Log-Likelihood = -296989.16526315006
epoch 1000; Log-Likelihood = -297541.6484492975
0.5
epoch 1; Log-Likelihood = -241267.75467613363
epoch 101; Log-Likelihood = -287905.5595721299
epoch 201; Log-Likelihood = -296036.0158372555
epoch 301; Log-Likelihood = -296889.67514056456
epoch 401; Log-Likelihood = -297419.28238957486
epoch 501; Log-Likelihood = -299178.10845163127
epoch 601; Log-Likelihood = -300267.2646821964
epoch 701; Log-Likelihood = -301587.56797620276
epoch 801; Log-Likelihood = -303057.96155298466
epoch 901; Log-Likelihood = -304615.805852

In [3]:
TrajectoryData = pd.DataFrame()
#TrajectoryData['Epoch'] = np.arange(3000)
for idx,lr in enumerate(LearningRates):
    TrajectoryData[f'lr= {lr}'] = Likelihoods[idx,:]

fig=px.line(TrajectoryData,
color_discrete_sequence=px.colors.qualitative.G10
)
fig.update_layout(
    title="Learning Rate: Hidden Markov Model",
    width=800,height=600,
    legend={'title':"Learning Rates"},
    template="plotly_white",
)
fig.update_yaxes(
    title="log-Likelihood"
)
fig.update_xaxes(
    title="Epoch"
)

fig.show()